# Differentiable Programming In Swift Intro

This section gives a brief background into the vector-Jacobian product formulation for automatic differentiation, and how it works in Swift. Feel free to sim this section if you already have a good understanding of similar automatic differentiation systems, such as [JAX](https://github.com/google/jax). If you would like a lecture covering these materials, please check out the Fast.AI series (TODO: insert link).

In order to understand how automatic differentiation works, we assume relatively little calculus background. The key pieces of information to know are:

 - The derivative of $x^2$, which is: $\frac{d}{dx} x^2 = 2x \frac{d}{dx}$
 - The derivative of $sin(x)$, which is: $\frac{d}{dx} \sin(x) = \cos(x) \frac{d}{dx}$
 - The derivative of $x + x$, which is $\frac{d}{dx}(x + x) = 2\frac{d}{dx}$.
 - The chain rule, which allows composition of differentiable functions: $$\frac{d}{dx}\sin(x^2) = \cos(x^2) \cdot 2x \cdot \frac{d}{dx}$$ and in its general form can be written as: $$\frac{d}{dx}\left[ f(g(x)) \right] = f'(g(x))g'(x) \frac{d}{dx}$$ where $f'$ and $g'$ correspond to the derivatives of functions $f$ and $g$ respectively with respect to their arguments.

## A trivial example

S4TF's automatic differentiation system is a "source-to-source" system, where source code for the function to be differentiated will be transformed (by the compiler) into the source code that computes the derivative. In order to understand what's going on, we're going to start by writing everything out explicitly.

As a warm up, we start with the trivial example: $x^2$. We can represent this in code as follows:

> Note: recall that $\frac{d}{dx} x^2 = 2x$

In [1]:
func square(_ x: Float) -> Float {
    return x * x
}

func squareDerivative(_ x: Float) -> Float {
    return 2 * x
}

## Compositions of functions

Simple polynomials are the easy case. Let's take the derivative of a more complicated function: $\sin(x^2)$, whose derivative is $\cos(x^2) \cdot 2x$ (recall the chain rule). We can write this in code as follows:

In [2]:
import Glibc

func myFunc(_ x: Float) -> Float {
    return sin(x * x)
}
func myFuncDerivative(_ x: Float) -> Float {
    return 2 * x * cos(x * x)
}

## A slightly more efficient implementation

Looking at the chain rule and our derivative implementation above, we notice that there's redundant computation going on. Concretely, in both `myFunc` and `myFuncDerivative`, we compute `x * x`. (From the chain rule definition, this is $g(x)$.) In the general case, `x * x` could be extremely expensive and we often want to only do this computation once.

We can thus rewrite our function and its derivative as follows and use a closure to capture the intermediates:

In [3]:
func myFuncBetter(_ x: Float) -> (value: Float, backward: () -> Float) {
    let xSquared = x * x
    let value = sin(xSquared)
    let backwards = { 2 * x * cos(xSquared) }  // A closure that captures `xSquared`
    return (value, backwards)
}

## Fully general derivatives

We've been a bit sloppy in our mathematics, but we're getting away with it because `Float` is a scalar. But if we were to generalize to more complicated types, we have to remember the $\frac{d}{dx}$ part of our derivatives.

The following leap allows us to "multiply" by the $\frac{d}{dx}$ correctly:

In [4]:
func myFuncCorrect(_ x: Float) -> (value: Float, deriv: (Float) -> Float) {
    let xSquared = x * x
    let value = sin(xSquared)
    let deriv = { (v: Float) -> Float in
        let gradXSquared = v * cos(xSquared)
        let gradX = gradXSquared * 2 * x
        return gradX
    }
    return (value: value, deriv: deriv)
}

In this formulation, the `deriv` function is technically a ["pullback" from differential geometry](https://en.wikipedia.org/wiki/Pullback_(differential_geometry)). We will use that term-of-art for the remainder of this tutorial.

## Rewrite using pullbacks

This pullback formulation is convenient, because we can now compose it together in a very regular (read automatable) manner.

In [5]:

func sinWithPullback(_ x: Float) -> (value: Float, pullback: (Float) -> Float) {
    return (sin(x), { dx in cos(x) * dx })
}

func squareWithPullback(_ x: Float) -> (value: Float, pullback: (Float) -> Float) {
    return (x * x, { dx in 2 * x * dx })
}

func myFuncPullbacks(_ x: Float) -> (value: Float, pullback: (Float) -> Float) {
    let (xSquared, pb1) = squareWithPullback(x)
    let (value, pb2) = sinWithPullback(xSquared)
    return (value, { dx in
        let gradXSquared = pb2(dx)
        let gradX = pb1(gradXSquared)
        return gradX
    })
}

Note: be sure to carefully read through the code to convince yourself that this new structure results in the exact same computation!

> Recall that in the end: $\frac{d}{dx}$ of the scalar $x$ is just 1, which is what we use to feed into the pullback.

We can use this new formulation as follows:

In [6]:
let x: Float = 1.3
let (value, pb) = myFuncPullbacks(x)
print("The value of the function at \(x) is: \(value)")
print("The derivative at \(x) is: \(pb(1))")

The value of the function at 1.3 is: 0.99290365
The derivative at 1.3 is: -0.3091956


## Generalizing to arbitrary expressions

Up until this point, we've been handwriting the derivatives for specific functions. But we now have a formulation that is regular and composible. (In fact, it is so regular, we can make the computer write the backwards pass for us! aka automatic differentiation.) The rules are:

1. Rewrite every expression in the forward pass into a form that computes the value like normal, and also produces an additional deriv function.
2. Construct a backwards pass that threads the derivs together in the reverse order.

In an abstract form, we transform a function that looks like:

```swift
func myFunction(_ arg: Float) -> Float {
    let tmp1 = expression1(arg)
    let tmp2 = expression2(tmp1)
    let tmp3 = expression3(tmp2)
    return tmp3
}
```

into a function that looks like this:

```swift
func myFunctionValueWithPullback(_ arg: Float) -> (value: Float, pullback: (Float) -> Float) {
    let (tmp1, pb1) = expression1ValueWithPullback(arg)
    let (tmp2, pb2) = expression2ValueWithPullback(tmp1)
    let (tmp3, pb3) = expression3ValueWithPullback(tmp2)
    return (value: tmp3,
            pullback: { dArg in
                let grad2 = pb3(dArg)
                let grad1 = pb2(grad2)
                let gradArg = pb1(grad1)
                return gradArg
    })
}
```

The Swift compiler does this exact transformation to all functions marked with `@differentiable` (when an explicit derivative is not specified)!

## Generalizing beyond unary functions

Up until now, we have been using functions that don't "reuse" values in their computation. Our running example of $\frac{d}{dx}\sin(x^2)$ is too simple. Let's make it a bit more complicated and use $\frac{d}{dx}\sin(x^2) + x$ as our motivating expression going forward. The new additional function `+` is a binary function (whereas all our previous functions were unary functions).

From mathematics, we know the derivative should be: $$\frac{d}{dx} \sin(x^2) + x = (2x \cos(x^2) + 1)\frac{d}{dx}$$

We will now write out the pullbacks by hand for illustrative purposes (pay attention to the pullback for the `+` function)!

In [7]:
func myComplexFunction(_ x: Float) -> Float {
    let tmp1 = square(x)
    let tmp2 = sin(tmp1)
    let tmp3 = tmp2 + x
    return tmp3
}

func plusWithPullback(_ x: Float, _ y: Float) -> (value: Float, pullback: (Float) -> (Float, Float)) {
    return (x + y, { d in (d, d) })  // Value semantics are great! ;-)
}

In [8]:
func myComplexFunctionWithPullback(_ x: Float) -> (value: Float, pullback: (Float) -> Float) {
    let (tmp1, pb1) = squareWithPullback(x)
    let (tmp2, pb2) = sinWithPullback(tmp1)
    let (tmp3, pb3) = plusWithPullback(tmp2, x)  // pb3 has type (Float) -> (Float, Float)
    return (tmp3, { dx in
        // Initialize all the gradients for all values at zero.
        var gradX = Float(0.0)
        var grad1 = Float(0.0)
        var grad2 = Float(0.0)
        var grad3 = Float(0.0)
        
        // Add the temporaries to the gradients as we run the backwards pass.
        grad3 += dx
        let (tmp2, tmpXa) = pb3(grad3)
        grad2 += tmp2
        gradX += tmpXa
        let tmp1 = pb2(grad2)
        grad1 += tmp1
        let tmpXb = pb1(grad1)
        gradX += tmpXb

        // Return the computed gradient.
        return gradX
    })
}

In [9]:
// Test it out:
print(myComplexFunction(2.0))  // Print the value of our function.
let (tmp, pb) = myComplexFunctionWithPullback(2.0)
print(tmp)  // Expect the same value as the first print!
print(pb(1)) // Print the derivative of the function at 2.000
print((myComplexFunction(2.001) - myComplexFunction(1.999)) / 0.002)  // Approximate the derivative at 2.0

1.2431974
1.2431974
-1.6145744
-1.6145109


Note: Swift's automatic differentiation system is more sophisticated, because we want to handle non-scalar values and beyond. As a result, we have separate types to represent the `Tangent` and `Cotangent` which I'm not covering here, but is important when not everything is a `Tensor`!

# AD of Array Access

This section explores how Swift's value semantics enable efficient differentiation of array indexing.

We will use the following program as a representative example:

In [10]:
// Forward:
func myOp(values: [Float], a: Int, b: Int) -> Float {
    return values[a] + values[b]
}

## A functional representation (idealized)

We will now write out an idealized version of a pullback for `myOp`:

In [11]:
// Pullback, functional style (idealized)
func myOpWithPullback(values: [Float], a: Int, b: Int) -> (value: Float, pullback: (Float) -> [Float]) {
    return (values[a] + values[b], { dx in
        var zeros = Array(repeating: Float(0), count: values.count)
        // Note: must += instead of assign to handle aliasing!
        zeros[a] += dx
        zeros[b] += dx
        return zeros
    })
}

## Reality Check

The above is exactly what you'd want to have happen. Unfortunately, that doesn't work with our _functional_ automatic differentiation formulation. Below is what actually would execute in a functional formulation:

In [12]:
func subscriptWithPullback(_ values: [Float], at index: Int) -> (value: Float, pullback: (Float) -> [Float]) {
    let size = values.count  // Optimization: capture only the size necessary instead of the whole array.
    return (values[index], { dx in
        var tmp = Array(repeating: Float(0), count: size)
        tmp[index] = dx
        return tmp
    })
}

func sumArraysHelper(_ a: [Float], _ b: [Float]) -> [Float] {
    // Written out explicitly
    precondition(a.count == b.count)
    var result = Array(repeating: Float(0), count: a.count)
    for i in 0..<a.count {
        result[i] = a[i] + b[i]
    }
    return result
}

func myOpWithPullbackFunctional(values: [Float], a: Int, b: Int) -> (value: Float, pullback: (Float) -> [Float]) {
    let (aVal, pbA) = subscriptWithPullback(values, at: a)
    let (bVal, pbB) = subscriptWithPullback(values, at: b)
    let result = aVal + bVal
    return (result, { dx in
        var dValues = Array(repeating: Float(0), count: values.count)
        let dA = pbA(dx)
        dValues = sumArraysHelper(dValues, dA)
        let dB = pbB(dx)
        dValues = sumArraysHelper(dValues, dB)
        return dValues
    })
}

The end result is that in the functional formulation, we create 3 arrays of zero's, and we have to do an expensive sum across whole arrays that could be very expensive. As Dougal highlights effectively in his presentation, a functional approach to automatic differention of array subscripts violates the principle of reverse mode AD that the backwards pass should have approximately the same cost as the forwards pass.

> Note: with careful optimization, you can avoid one of the zero materializations, and one of the calls to `sumArraysHelper`, but the fundamental principles remain.

## A value-semantic formulation

We now see how applying the principles of value semantics allow us an efficient formulation of the backwards pass!

In [13]:
func subscriptWithPullbackValueSemantics(
    _ values: [Float],
    at index: Int
) -> (value: Float, pullback: (Float, inout [Float]) -> Void) {
    (values[index], { dx, dValues in
        dValues[index] += dx
    })
}

func myOpWithPullback(values: [Float], a: Int, b: Int) -> (value: Float, pullback: (Float, inout [Float]) -> Void) {
    let (aVal, pbA) = subscriptWithPullbackValueSemantics(values, at: a)
    let (bVal, pbB) = subscriptWithPullbackValueSemantics(values, at: b)
    let result = aVal + bVal
    return (result, { dx, dValues in
        pbA(dx, &dValues)
        pbB(dx, &dValues)
    })
}

## Composing value semantics

An astute reader will note that our type signature for the pullback of `myOpWithPullback` could be improved. In particular, we should further take the `dValues` array as `inout` instead of allocating and returning the zeros. This composes perfectly and would yield an efficient derivative of a following hypothetical function (although writing out the proper pullback for both `myOp` and `myCompositeOp` is left as an exercise for the reader).

In [14]:
func myCompositeOp(_ values: [Float]) -> Float {
    var pairSum: Float = 0
    for i in 0..<(values.count / 2) {
        pairSum += myOp(values: values, a: i, b: i + 1)
    }
    return pairSum
}

_But aren't you just pushing the problem elsewhere instead of actually solving it?_ Of course a data structure to hold the derivative must be allocated at some point. But ideally it is allocated exactly once. Yes, somewhere some function must allocate the value that kicks things off, but everything else simply accumulates into it efficiently.

## Applications beyond array indexing

Leveraging `inout` for in-place mutation applies not just to the problem of array indexing, but also to slicing out views into tensors, and even accessing subcomponents of larger differentiable types (if the automatic differentiation system is general purpose enough to support non-hyper-rectangular differentiable data types) such as graphs, trees, or other data structures.

Further examples are left as an exercise to the reader. ;-)